In [1]:
!pip install --quiet openai

     |████████████████████████████████| 44 kB 1.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 146 kB 7.0 MB/s 


In [2]:
import os
import openai
import getpass

openai.api_key = getpass.getpass(prompt='Enter OpenAI API key:')



Enter OpenAI API key:··········


## Generate paraphrases with GPT-3

In [9]:
response = openai.Completion.create(
  engine="davinci-instruct-beta",
  prompt="I am a diverse paraphrasing expert. I will generate many random pairs of original and paraphrased sentences.\n\nHere are a few:\n\noriginal: Once, a group of frogs was roaming around the forest in search of water.\nparaphrase:  A gang of frogs was roaming about the woods in search of water once more.\n\noriginal:",
  temperature=0.7,
  max_tokens=100,
  top_p=0.9,
  frequency_penalty=0.5,
  presence_penalty=0.6,
  stop=["original"]
)



In [10]:
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " The few times in my life I've seen a ghost, the ghost has been white.\nparaphrase:  The few times in my life that I have seen a ghost, the ghost has been white.\n\n"
    }
  ],
  "created": 1670134967,
  "id": "cmpl-6JcghBp101hoWo3OBrkDCVj88zwnI",
  "model": "davinci-instruct-beta",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 45,
    "prompt_tokens": 76,
    "total_tokens": 121
  }
}


In [11]:
output_string = response['choices'][0]['text'].strip()
output_modified = output_string.replace("paraphrase:","")
original_sentence, paraphrased_sentence = output_modified.split('\n')
original_sentence = original_sentence.strip()
paraphrased_sentence = paraphrased_sentence.strip()
print ("-----------------------------------------------------------------------")
print (original_sentence)
print (paraphrased_sentence)

-----------------------------------------------------------------------
The few times in my life I've seen a ghost, the ghost has been white.
The few times in my life that I have seen a ghost, the ghost has been white.


## Streamlit Dataset Creator UI

In [12]:
!pip install --quiet streamlit==1.1.0  

     |████████████████████████████████| 8.3 MB 3.8 MB/s 
     |████████████████████████████████| 182 kB 102.8 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████████████████████████████| 4.7 MB 81.9 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 


In [13]:
%%writefile app.py
import pandas as pd
import streamlit as st
import json
import streamlit.components.v1 as components
import os
import openai
import getpass
import pandas as pd

# openai.api_key = getpass.getpass(prompt='Enter OpenAI API key:')
openai.api_key  = st.text_input("OpenAI API Key", '',type="password")

# st.write('# Moviepro.ai')
st.markdown("<h1 style='text-align: center; color: blue;'>Paraphrase Dataset Generator</h1>", unsafe_allow_html=True)
st.write("## Create paraphrase dataset, powered by GPT-3.")

st.write("")
if "original" not in st.session_state:
    st.session_state.original = ""
if "paraphrase" not in st.session_state:
    st.session_state.paraphrase = ""
if "df" not in st.session_state:
    st.session_state.df = pd.DataFrame(columns=['original','paraphrase'])

def getparaphrase():
  response = openai.Completion.create(
  engine="davinci-instruct-beta",
  prompt="I am a diverse paraphrasing expert. I will generate many random pairs of original and paraphrased sentences.\n\nHere are a few:\n\noriginal: Once, a group of frogs was roaming around the forest in search of water.\nparaphrase:  A gang of frogs was roaming about the woods in search of water once more.\n\noriginal:",
  temperature=0.7,
  max_tokens=100,
  top_p=0.9,
  frequency_penalty=0.5,
  presence_penalty=0.6,
  stop=["original"]
  )
  output_string = response['choices'][0]['text'].strip()
  output_modified = output_string.replace("paraphrase:","")
  original_sentence, paraphrased_sentence = output_modified.split('\n')
  st.session_state.original = original_sentence.strip()
  st.session_state.paraphrase = paraphrased_sentence.strip()

def addtodb():
  data = {
    'original': st.session_state.original,
    'paraphrase': st.session_state.paraphrase
    }

  st.session_state.df = st.session_state.df.append(data, ignore_index=True)
  st.write (st.session_state.df.shape)






st.button("Generate paraphrases",on_click=getparaphrase)
st.button("Add to Database",on_click=addtodb)

st.write("**Original:** "+st.session_state.original)
st.write("**Paraphrase:** "+st.session_state.paraphrase)
st.dataframe(st.session_state.df)

Writing app.py


In [14]:
!streamlit run app.py & npx localtunnel --port 8501

2022-12-04 06:26:01.019 INFO    numexpr.utils: NumExpr defaulting to 8 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.134.91:8501

npx: installed 22 in 2.546s
your url is: https://chilly-kids-juggle-35-229-134-91.loca.lt
2022-12-04 06:26:42.089 message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/davinci-instruct-beta/completions
2022-12-04 06:26:44.663 message='OpenAI API response' path=https://api.openai.com/v1/engines/davinci-instruct-beta/completions processing_ms=1845 request_id=8ef3e12107bba83251c4e9679f1163ed response_code=200
2022-12-04 06:27:08.381 message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/davinci-instruct-beta/completions
2022-12-04 06:27:10.420 message='OpenAI API response' path=https://api.openai.com/v1/engines/davinci-instruct-beta/completions processing_ms=1416 request_id=7976582e6343270c59c3c63bf909f265 response_co